In [1]:
#Import numpy library for standard deviation calculations
import numpy as np

In [2]:
#Read lines from Zeek network logs
f = open('asyncrat_conn.log', 'r')
lines = f.readlines()
f.close()

In [3]:
#Parse Zeek network logs to identify unique source/destination pairs
logs = []
for line in lines[9:]:
    fields = line.split('\t')
    src = fields[2]
    dst = fields[4]
    logs.append(src + ' ' + dst)
src_dst_pairs = list(set(logs))

In [4]:
#Gather the times of connections for each unique source/destination pair, ignoring network connections to local network
times = {}
for pair in src_dst_pairs:
    times[pair] = []

for key in times.keys():
    values = key.split(' ')
    src = values[0]
    dst = values[1]
    
    if '192.168' in dst:
        continue
    elif '::' in dst:
        continue
    elif '239.255' in dst:
        continue
    elif '224.0.0' in dst:
        continue
    elif '255.255' in dst:
        continue
    for line in lines[9:]:
        if src in line and dst in line:
            times[key].append(float(line.split('\t')[0]))

#Calculate the times between connections for each unique source/destination pair
diffs = {}
for key in times.keys():
    diffs[key] = []
    
for key in diffs.keys():
    k = 1
    if len(times[key]) < 5:
        continue
    for value in times[key][1:]:
        diff = value - times[key][k-1]
        diffs[key].append(diff)
        k = k + 1

#Calculate standard deviations for each unique source/destination pair
stds = {}
for key in diffs.keys():
    a = np.array(diffs[key])           
    if len(a) < 4:
        continue
        
    std = np.std(a)
    stds[std] = key

#Save list of calculated standard deviations to numpy array
np_stds = np.array(list(stds.keys()))

In [5]:
#Print out all standard deviations below the 5th percentile, which should contain low standard deviations
for key in stds.keys():
    if key < np.percentile(np_stds, 5):
        print('SRC DST: ' + stds[key] + ', standard deviation: ' + str(key))

SRC DST: 192.168.100.151 184.150.58.155, standard deviation: 0.0071574957538394855
SRC DST: 192.168.100.139 162.159.200.1, standard deviation: 215.64447080534612
SRC DST: 192.168.100.150 20.119.174.243, standard deviation: 21.33984602366211
SRC DST: 192.168.100.136 143.198.3.13, standard deviation: 16.458849041232124
SRC DST: 192.168.100.136 172.208.51.75, standard deviation: 0.868396098873865


In [6]:
len(times['192.168.100.151 184.150.58.155'])

5

In [7]:
len(times['192.168.100.136 172.208.51.75'])

13281